In [1]:
# imports
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import seaborn as sns
plt.rcParams['figure.figsize'] = (16, 8)

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# this allows plots to appear directly in the notebook
%matplotlib inline

google_api_key = %env GOOGLE_API_KEY

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.models.glyphs import Patches
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap, figure

google_api_key = %env GOOGLE_API_KEY

output_file("patch.html")

In [2]:
def create_polygon(mp):
    try:
        xs = []
        ys = []
        for e in mp[16:-3].split(', '):
            pt = [float(x) for x in e.split(' ')]
            xs.append(pt[0])
            ys.append(pt[1])
        return xs,ys
    except:
        return 0



In [3]:
# read in data
sold = pd.read_csv('data/detroit_tax_auction_2017.csv', index_col='parcel_id')
sold.drop(labels=['Unnamed: 14','px_per_foot'],axis=1,inplace=True)

# replace missing with median values
sold['sqft'] = sold.sqft.replace(0, 4400)
sold['year_built'] = sold.year_built.replace(0, 1938)
sold['px_per_foot'] = sold.winning_bid / sold.sqft
sold['tax_value_per_foot'] = sold.taxable_value / sold.sqft
sold.zipcode = sold.zipcode.astype("str")
sold.year_built = sold.year_built.fillna(1900).astype("int64")

# prepare dataset for mapping
sold.geometry = sold.geometry.apply(create_polygon)

In [4]:
sold.lon = sold.geometry.apply(lambda x: x[0] if x != 0 else 0)
sold.lat = sold.geometry.apply(lambda x: x[1] if x != 0 else 0)

In [5]:
sold.lat.iloc[35:37].values

array([ [42.39054152988467, 42.3906755931077, 42.390528658902824, 42.39039549149306, 42.3904680477823, 42.39054152988467],
       [42.390190783275436, 42.39011227534397, 42.38970051943078, 42.38977902739197, 42.390190783275436]], dtype=object)

In [6]:
# map_options = GMapOptions(lat=42.39054152988467, lng=-82.99644723680774, map_type="roadmap", zoom=12)
# p = gmap(google_api_key, map_options, title="Detroit")
p = figure()

source = ColumnDataSource(
    data=dict(lat=sold.lat.values,
              lon=sold.lon.values)
)

glyph = Patches(xs="lon", ys="lat", fill_color="#fb9a99")
p.add_glyph(source, glyph)

show(p)